# MobileNetV3 Variation Impl
- MobileNetV3의 Large/Small 모델과, 각 모델에서의 아래와 같은 설정으로 테스트를 진행해본다.
    - width_multiplier: 0.35, 0.5, 0.75, 1.0
    - input_resolution: 96, 128, 160, 192, 224, 256

In [2]:
import torch
from torch import nn

In [3]:
class SqueezeExciteModule(nn.Module):
    def __init__(self, expand_size):
        super(SqueezeExciteModule, self).__init__()

        self.se_0_0 = nn.AdaptiveAvgPool2d(output_size=1)
        self.se_0_1 = nn.Flatten()

        self.se_1_0 = nn.Linear(in_features=expand_size, out_features=expand_size)
        self.se_1_1 = nn.ReLU(inplace=True)

        self.se_2_0 = nn.Linear(in_features=expand_size, out_features=expand_size)
        self.se_2_1 = nn.Hardsigmoid(inplace=True)

    def forward(self, x):
        x = self.se_0_0(x)
        x = self.se_0_1(x)

        x = self.se_1_0(x)
        x = self.se_1_1(x)

        x = self.se_2_0(x)
        x = self.se_2_1(x)
        x = torch.unsqueeze(x, -1)
        x = torch.unsqueeze(x, -1)
        return x


class Bottleneck(nn.Module):
    def __init__(self, in_channels, out_channels, dw_kernel_size, expand_size, squeeze_excite,
                 nonlinearity, stride, bias = True):
        super(Bottleneck, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.expand_size = expand_size
        self.squeeze_excite = squeeze_excite
        self.stride = stride
        self.dw_kernel_size = dw_kernel_size
        self.bias = bias

        if nonlinearity == 'hardswish':
            self.Nonliearity = nn.Hardswish
        elif nonlinearity == 'relu':
            self.Nonliearity = nn.ReLU
        else:
            raise RuntimeError("No such nonlinearity!")

        # 1x1 Conv2d + NL
        self.bottleneck_0_0 = nn.Conv2d(in_channels=in_channels, out_channels=expand_size, kernel_size=(1, 1),
                                        bias=self.bias)
        self.bottleneck_0_1 = nn.BatchNorm2d(num_features=expand_size)
        self.bottleneck_0_2 = self.Nonliearity(inplace=True)

        # Dwise + NL
        self.bottleneck_1_0 = nn.Conv2d(in_channels=expand_size, out_channels=expand_size,
                                        kernel_size=self.dw_kernel_size,
                                        stride=self.stride, padding=self.dw_kernel_size[0] // 2, groups=expand_size,
                                        bias=self.bias)
        self.bottleneck_1_1 = nn.BatchNorm2d(num_features=expand_size)

        # Squeeze-Excite
        if self.squeeze_excite:
            self.squeeze_excite_0 = SqueezeExciteModule(
                expand_size=expand_size
            )
        else:
            self.squeeze_excite_0 = nn.Identity()

        # Final 1x1 Conv2d
        self.bottleneck_final_0 = nn.Conv2d(in_channels=expand_size, out_channels=out_channels, kernel_size=(1, 1),
                                            bias=self.bias)
        self.bottleneck_final_1 = nn.BatchNorm2d(num_features=out_channels)

        # Downsampling first layer
        self.bottleneck_final_2 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                                            kernel_size=(1, 1), stride=self.stride, bias=self.bias)

    def forward(self, x):
        x_0 = self.bottleneck_0_0(x)
        x_0 = self.bottleneck_0_1(x_0)
        x_0 = self.bottleneck_0_2(x_0)

        x_0 = self.bottleneck_1_0(x_0)
        x_0 = self.bottleneck_1_1(x_0)

        x_1 = self.squeeze_excite_0(x_0)
        x_0 = x_0 * x_1

        x_0 = self.bottleneck_final_0(x_0)
        x_0 = self.bottleneck_final_1(x_0)
        x_b = self.bottleneck_final_2(x)
        return x_0.add(x_b)

In [4]:
class MobileNetV3(nn.Module):
    def __init__(self, size='large', width_mult=1.0, classifier=True, classifier_out_features=1000, bias=True, dropout=0.2):
        super(MobileNetV3, self).__init__()
        
        self.bias = bias
        self.dropout = dropout
        
        if size.lower() == 'large':
            conv_0_0 = nn.Conv2d(in_channels=3, out_channels=int(16 * width_mult),
                                 kernel_size=(3, 3), stride=2, padding=3 // 2, bias=self.bias)
            conv_0_1 = nn.BatchNorm2d(num_features=int(16 * width_mult))
            conv_0_2 = nn.Hardswish(inplace=True)
            
            conv_1_0 = Bottleneck(in_channels=int(16 * width_mult),
                                  out_channels=int(16 * width_mult), dw_kernel_size=(3, 3),
                                  expand_size=16, squeeze_excite=False, nonlinearity='relu', stride=1, bias=self.bias)
            conv_2_0 = Bottleneck(in_channels=int(16 * width_mult),
                                  out_channels=int(24 * width_mult), dw_kernel_size=(3, 3),
                                  expand_size=64, squeeze_excite=False, nonlinearity='relu', stride=2, bias=self.bias)
            conv_3_0 = Bottleneck(in_channels=int(24 * width_mult),
                                  out_channels=int(24 * width_mult), dw_kernel_size=(3, 3),
                                  expand_size=72, squeeze_excite=False, nonlinearity='relu', stride=1, bias=self.bias)
            conv_4_0 = Bottleneck(in_channels=int(24 * width_mult),
                                  out_channels=int(40 * width_mult), dw_kernel_size=(5, 5),
                                  expand_size=72, squeeze_excite=True, nonlinearity='relu', stride=2, bias=self.bias)
            conv_5_0 = Bottleneck(in_channels=int(40 * width_mult),
                                  out_channels=int(40 * width_mult), dw_kernel_size=(5, 5),
                                  expand_size=120, squeeze_excite=True, nonlinearity='relu', stride=1, bias=self.bias)
            conv_6_0 = Bottleneck(in_channels=int(40 * width_mult),
                                  out_channels=int(40 * width_mult), dw_kernel_size=(5, 5),
                                  expand_size=120, squeeze_excite=True, nonlinearity='relu', stride=1, bias=self.bias)
            conv_7_0 = Bottleneck(in_channels=int(40 * width_mult),
                                  out_channels=int(80 * width_mult), dw_kernel_size=(3, 3),
                                  expand_size=240, squeeze_excite=False, nonlinearity='hardswish', stride=2, bias=self.bias)
            conv_8_0 = Bottleneck(in_channels=int(80 * width_mult),
                                  out_channels=int(80 * width_mult), dw_kernel_size=(3, 3),
                                  expand_size=200, squeeze_excite=False, nonlinearity='hardswish', stride=1, bias=self.bias)
            conv_9_0 = Bottleneck(in_channels=int(80 * width_mult),
                                  out_channels=int(80 * width_mult), dw_kernel_size=(3, 3),
                                  expand_size=184, squeeze_excite=False, nonlinearity='hardswish', stride=1, bias=self.bias)
            conv_10_0 = Bottleneck(in_channels=int(80 * width_mult),
                                  out_channels=int(80 * width_mult), dw_kernel_size=(3, 3),
                                  expand_size=184, squeeze_excite=False, nonlinearity='hardswish', stride=1, bias=self.bias)
            conv_11_0 = Bottleneck(in_channels=int(80 * width_mult),
                                  out_channels=int(112 * width_mult), dw_kernel_size=(3, 3),
                                  expand_size=480, squeeze_excite=True, nonlinearity='hardswish', stride=1, bias=self.bias)
            conv_12_0 = Bottleneck(in_channels=int(112 * width_mult),
                                  out_channels=int(112 * width_mult), dw_kernel_size=(3, 3),
                                  expand_size=672, squeeze_excite=True, nonlinearity='hardswish', stride=1, bias=self.bias)
            conv_13_0 = Bottleneck(in_channels=int(112 * width_mult),
                                  out_channels=int(160 * width_mult), dw_kernel_size=(5, 5),
                                  expand_size=672, squeeze_excite=True, nonlinearity='hardswish', stride=2, bias=self.bias)
            conv_14_0 = Bottleneck(in_channels=int(160 * width_mult),
                                  out_channels=int(160 * width_mult), dw_kernel_size=(5, 5),
                                  expand_size=960, squeeze_excite=True, nonlinearity='hardswish', stride=1, bias=self.bias)
            conv_15_0 = Bottleneck(in_channels=int(160 * width_mult),
                                  out_channels=int(160 * width_mult), dw_kernel_size=(5, 5),
                                  expand_size=960, squeeze_excite=True, nonlinearity='hardswish', stride=1, bias=self.bias)
            
            conv_16_0 = nn.Conv2d(in_channels=int(160 * width_mult), out_channels=int(960 * width_mult),
                                 kernel_size=(1, 1), stride=1, bias=self.bias)
            conv_16_1 = nn.BatchNorm2d(num_features=int(960 * width_mult))
            conv_16_2 = nn.Hardswish(inplace=True)
            
            self.features = nn.Sequential(
                conv_0_0,
                conv_0_1,
                conv_0_2,
                conv_1_0,
                conv_2_0,
                conv_3_0, 
                conv_4_0, 
                conv_5_0, 
                conv_6_0, 
                conv_7_0, 
                conv_8_0, 
                conv_9_0, 
                conv_10_0,
                conv_11_0,
                conv_12_0,
                conv_13_0,
                conv_14_0,
                conv_15_0,
                conv_16_0,
                conv_16_1,
                conv_16_2
            )
            
            if classifier:
                self.classifiers = nn.Sequential(
                    nn.AdaptiveAvgPool2d(output_size=1),
                    nn.Flatten(start_dim=1),
                    nn.Linear(int(960 * width_mult), int(ㅊ1280 * width_mult)),
                    nn.Dropout(p=self.dropout),
                    nn.Linear(int(1280 * width_mult), classifier_out_features)
                )
            else:
                self.classifiers = nn.Sequential(
                    nn.BatchNorm2d(num_features=int(960 * width_mult)),
                    nn.Hardswish(),
                    nn.Conv2d(in_channels=int(960 * width_mult), out_channels=int(1280 * width_mult),
                                          kernel_size=(1, 1), bias=self.bias),
                    nn.Hardswish()
                )
            
        elif size.lower() == 'small':
            conv_0_0 = nn.Conv2d(in_channels=3, out_channels=int(16 * width_mult),
                                 kernel_size=(3, 3), stride=2, padding=3 // 2, bias=self.bias)
            conv_0_1 = nn.BatchNorm2d(num_features=int(16 * width_mult))
            conv_0_2 = nn.Hardswish()

            conv_1_0 = Bottleneck(in_channels=int(16 * width_mult),
                                  out_channels=int(16 * width_mult), dw_kernel_size=(3, 3),
                                  expand_size=16, squeeze_excite=True, nonlinearity='relu', stride=2, bias=self.bias)
            conv_2_0 = Bottleneck(in_channels=int(16 * width_mult),
                                  out_channels=int(24 * width_mult), dw_kernel_size=(3, 3),
                                  expand_size=72, squeeze_excite=False, nonlinearity='relu', stride=2, bias=self.bias)
            conv_3_0 = Bottleneck(in_channels=int(24 * width_mult),
                                  out_channels=int(24 * width_mult), dw_kernel_size=(3, 3),
                                  expand_size=88, squeeze_excite=False, nonlinearity='relu', stride=1, bias=self.bias)
            conv_4_0 = Bottleneck(in_channels=int(24 * width_mult),
                                  out_channels=int(40 * width_mult), dw_kernel_size=(5, 5),
                                  expand_size=96, squeeze_excite=True, nonlinearity='hardswish', stride=2, bias=self.bias)
            conv_5_0 = Bottleneck(in_channels=int(40 * width_mult),
                                  out_channels=int(40 * width_mult), dw_kernel_size=(5, 5),
                                  expand_size=240, squeeze_excite=True, nonlinearity='hardswish', stride=1, bias=self.bias)
            conv_6_0 = Bottleneck(in_channels=int(40 * width_mult),
                                  out_channels=int(40 * width_mult), dw_kernel_size=(5, 5),
                                  expand_size=240, squeeze_excite=True, nonlinearity='hardswish', stride=1, bias=self.bias)
            conv_7_0 = Bottleneck(in_channels=int(40 * width_mult),
                                  out_channels=int(48 * width_mult), dw_kernel_size=(5, 5),
                                  expand_size=120, squeeze_excite=True, nonlinearity='hardswish', stride=1, bias=self.bias)
            conv_8_0 = Bottleneck(in_channels=int(48 * width_mult),
                                  out_channels=int(48 * width_mult), dw_kernel_size=(5, 5),
                                  expand_size=144, squeeze_excite=True, nonlinearity='hardswish', stride=1, bias=self.bias)
            conv_9_0 = Bottleneck(in_channels=int(48 * width_mult),
                                  out_channels=int(96 * width_mult), dw_kernel_size=(5, 5),
                                  expand_size=288, squeeze_excite=True, nonlinearity='hardswish', stride=2, bias=self.bias)
            conv_10_0 = Bottleneck(in_channels=int(96 * width_mult),
                                   out_channels=int(96 * width_mult), dw_kernel_size=(5, 5),
                                   expand_size=576, squeeze_excite=True, nonlinearity='hardswish', stride=1, bias=self.bias)
            conv_11_0 = Bottleneck(in_channels=int(96 * width_mult),
                                   out_channels=int(96 * width_mult), dw_kernel_size=(5, 5),
                                   expand_size=576, squeeze_excite=True, nonlinearity='hardswish', stride=1, bias=self.bias)

            conv_12_0 = nn.Conv2d(in_channels=int(96 * width_mult), out_channels=int(576 * width_mult),
                                  kernel_size=(1, 1), bias=self.bias)
            conv_12_1 = nn.Hardswish()
            conv_12_2 = nn.BatchNorm2d(num_features=int(576 * width_mult))

            self.features = nn.Sequential(
                conv_0_0,
                conv_0_1,
                conv_0_2,
                conv_1_0,
                conv_2_0,
                conv_3_0,
                conv_4_0,
                conv_5_0,
                conv_6_0,
                conv_7_0,
                conv_8_0,
                conv_9_0,
                conv_10_0,
                conv_11_0,
                conv_12_0,
                conv_12_1,
                conv_12_2
            )

            if classifier:
                self.classifiers = nn.Sequential(
                    nn.AdaptiveAvgPool2d(output_size=1),
                    nn.Flatten(start_dim=1),
                    nn.Linear(int(576 * width_mult), int(1024 * width_mult)),
                    nn.Dropout(p=self.dropout),
                    nn.Linear(int(1024 * width_mult), classifier_out_features)
                )
            else:
                self.classifiers = nn.Sequential(
                    nn.BatchNorm2d(num_features=int(576 * width_mult)),
                    nn.Hardswish(),
                    nn.Conv2d(in_channels=int(576 * width_mult), out_channels=int(1280 * width_mult),
                                          kernel_size=(1, 1), bias=self.bias),
                    nn.Hardswish()
                )

    def forward(self, x):
        x = self.features(x)
        x = self.classifiers(x)
        return x

In [5]:
model = MobileNetV3(size='large', width_mult=1.0, classifier=False).eval()
model(torch.zeros(1, 3, 224, 224)).shape

torch.Size([1, 1280, 7, 7])

In [6]:
# Test libraries
# !pip3 uninstall -y torchsummary
# !pip3 install torchinfo
from torchinfo import summary

In [7]:
# Test MobileNetV3-large-1.0-224
summary(MobileNetV3(size='large', width_mult=1.0, classifier=False), input_size=(1, 3, 300, 300))

Layer (type:depth-idx)                        Output Shape              Param #
MobileNetV3                                   --                        --
├─Sequential: 1-1                             [1, 960, 10, 10]          --
│    └─Conv2d: 2-1                            [1, 16, 150, 150]         448
│    └─BatchNorm2d: 2-2                       [1, 16, 150, 150]         32
│    └─Hardswish: 2-3                         [1, 16, 150, 150]         --
│    └─Bottleneck: 2-4                        [1, 16, 150, 150]         --
│    │    └─Conv2d: 3-1                       [1, 16, 150, 150]         272
│    │    └─BatchNorm2d: 3-2                  [1, 16, 150, 150]         32
│    │    └─ReLU: 3-3                         [1, 16, 150, 150]         --
│    │    └─Conv2d: 3-4                       [1, 16, 150, 150]         160
│    │    └─BatchNorm2d: 3-5                  [1, 16, 150, 150]         32
│    │    └─Identity: 3-6                     [1, 16, 150, 150]         --
│    │    └─Conv2

In [6]:
# Test MobileNetV3-large-1.0-224
summary(MobileNetV3(size='large', width_mult=1.0, classifier=False), input_size=(1, 3, 224, 224))

Layer (type:depth-idx)                        Output Shape              Param #
MobileNetV3                                   --                        --
├─Sequential: 1-1                             [1, 960, 7, 7]            --
│    └─Conv2d: 2-1                            [1, 16, 112, 112]         448
│    └─BatchNorm2d: 2-2                       [1, 16, 112, 112]         32
│    └─Hardswish: 2-3                         [1, 16, 112, 112]         --
│    └─Bottleneck: 2-4                        [1, 16, 112, 112]         --
│    │    └─Conv2d: 3-1                       [1, 16, 112, 112]         272
│    │    └─BatchNorm2d: 3-2                  [1, 16, 112, 112]         32
│    │    └─ReLU: 3-3                         [1, 16, 112, 112]         --
│    │    └─Conv2d: 3-4                       [1, 16, 112, 112]         160
│    │    └─BatchNorm2d: 3-5                  [1, 16, 112, 112]         32
│    │    └─Identity: 3-6                     [1, 16, 112, 112]         --
│    │    └─Conv2

In [7]:
# Test MobileNetV3-large-1.0-192
summary(MobileNetV3(size='large', width_mult=1.0, classifier=False), input_size=(1, 3, 192, 192))

Layer (type:depth-idx)                        Output Shape              Param #
MobileNetV3                                   --                        --
├─Sequential: 1-1                             [1, 960, 6, 6]            --
│    └─Conv2d: 2-1                            [1, 16, 96, 96]           448
│    └─BatchNorm2d: 2-2                       [1, 16, 96, 96]           32
│    └─Hardswish: 2-3                         [1, 16, 96, 96]           --
│    └─Bottleneck: 2-4                        [1, 16, 96, 96]           --
│    │    └─Conv2d: 3-1                       [1, 16, 96, 96]           272
│    │    └─BatchNorm2d: 3-2                  [1, 16, 96, 96]           32
│    │    └─ReLU: 3-3                         [1, 16, 96, 96]           --
│    │    └─Conv2d: 3-4                       [1, 16, 96, 96]           160
│    │    └─BatchNorm2d: 3-5                  [1, 16, 96, 96]           32
│    │    └─Identity: 3-6                     [1, 16, 96, 96]           --
│    │    └─Conv2

In [8]:
# Test MobileNetV3-large-1.0-160
summary(MobileNetV3(size='large', width_mult=1.0, classifier=False), input_size=(1, 3, 160, 160))

Layer (type:depth-idx)                        Output Shape              Param #
MobileNetV3                                   --                        --
├─Sequential: 1-1                             [1, 960, 5, 5]            --
│    └─Conv2d: 2-1                            [1, 16, 80, 80]           448
│    └─BatchNorm2d: 2-2                       [1, 16, 80, 80]           32
│    └─Hardswish: 2-3                         [1, 16, 80, 80]           --
│    └─Bottleneck: 2-4                        [1, 16, 80, 80]           --
│    │    └─Conv2d: 3-1                       [1, 16, 80, 80]           272
│    │    └─BatchNorm2d: 3-2                  [1, 16, 80, 80]           32
│    │    └─ReLU: 3-3                         [1, 16, 80, 80]           --
│    │    └─Conv2d: 3-4                       [1, 16, 80, 80]           160
│    │    └─BatchNorm2d: 3-5                  [1, 16, 80, 80]           32
│    │    └─Identity: 3-6                     [1, 16, 80, 80]           --
│    │    └─Conv2

In [9]:
# Test MobileNetV3-large-1.0-128
summary(MobileNetV3(size='large', width_mult=1.0, classifier=False), input_size=(1, 3, 128, 128))

Layer (type:depth-idx)                        Output Shape              Param #
MobileNetV3                                   --                        --
├─Sequential: 1-1                             [1, 960, 4, 4]            --
│    └─Conv2d: 2-1                            [1, 16, 64, 64]           448
│    └─BatchNorm2d: 2-2                       [1, 16, 64, 64]           32
│    └─Hardswish: 2-3                         [1, 16, 64, 64]           --
│    └─Bottleneck: 2-4                        [1, 16, 64, 64]           --
│    │    └─Conv2d: 3-1                       [1, 16, 64, 64]           272
│    │    └─BatchNorm2d: 3-2                  [1, 16, 64, 64]           32
│    │    └─ReLU: 3-3                         [1, 16, 64, 64]           --
│    │    └─Conv2d: 3-4                       [1, 16, 64, 64]           160
│    │    └─BatchNorm2d: 3-5                  [1, 16, 64, 64]           32
│    │    └─Identity: 3-6                     [1, 16, 64, 64]           --
│    │    └─Conv2

In [10]:
# Test MobileNetV3-large-1.0-96
summary(MobileNetV3(size='large', width_mult=1.0, classifier=False), input_size=(1, 3, 96, 96))

Layer (type:depth-idx)                        Output Shape              Param #
MobileNetV3                                   --                        --
├─Sequential: 1-1                             [1, 960, 3, 3]            --
│    └─Conv2d: 2-1                            [1, 16, 48, 48]           448
│    └─BatchNorm2d: 2-2                       [1, 16, 48, 48]           32
│    └─Hardswish: 2-3                         [1, 16, 48, 48]           --
│    └─Bottleneck: 2-4                        [1, 16, 48, 48]           --
│    │    └─Conv2d: 3-1                       [1, 16, 48, 48]           272
│    │    └─BatchNorm2d: 3-2                  [1, 16, 48, 48]           32
│    │    └─ReLU: 3-3                         [1, 16, 48, 48]           --
│    │    └─Conv2d: 3-4                       [1, 16, 48, 48]           160
│    │    └─BatchNorm2d: 3-5                  [1, 16, 48, 48]           32
│    │    └─Identity: 3-6                     [1, 16, 48, 48]           --
│    │    └─Conv2

In [11]:
# Test MobileNetV3-Small-1.0-224
summary(MobileNetV3(size='small', width_mult=1.0, classifier=False), input_size=(1, 3, 224, 224))

Layer (type:depth-idx)                        Output Shape              Param #
MobileNetV3                                   --                        --
├─Sequential: 1-1                             [1, 576, 7, 7]            --
│    └─Conv2d: 2-1                            [1, 16, 112, 112]         448
│    └─BatchNorm2d: 2-2                       [1, 16, 112, 112]         32
│    └─Hardswish: 2-3                         [1, 16, 112, 112]         --
│    └─Bottleneck: 2-4                        [1, 16, 56, 56]           --
│    │    └─Conv2d: 3-1                       [1, 16, 112, 112]         272
│    │    └─BatchNorm2d: 3-2                  [1, 16, 112, 112]         32
│    │    └─ReLU: 3-3                         [1, 16, 112, 112]         --
│    │    └─Conv2d: 3-4                       [1, 16, 56, 56]           160
│    │    └─BatchNorm2d: 3-5                  [1, 16, 56, 56]           32
│    │    └─SqueezeExciteModule: 3-6          [1, 16, 1, 1]             544
│    │    └─Conv

In [12]:
# Test MobileNetV3-Small-1.0-192
summary(MobileNetV3(size='small', width_mult=1.0, classifier=False), input_size=(1, 3, 192, 192))

Layer (type:depth-idx)                        Output Shape              Param #
MobileNetV3                                   --                        --
├─Sequential: 1-1                             [1, 576, 6, 6]            --
│    └─Conv2d: 2-1                            [1, 16, 96, 96]           448
│    └─BatchNorm2d: 2-2                       [1, 16, 96, 96]           32
│    └─Hardswish: 2-3                         [1, 16, 96, 96]           --
│    └─Bottleneck: 2-4                        [1, 16, 48, 48]           --
│    │    └─Conv2d: 3-1                       [1, 16, 96, 96]           272
│    │    └─BatchNorm2d: 3-2                  [1, 16, 96, 96]           32
│    │    └─ReLU: 3-3                         [1, 16, 96, 96]           --
│    │    └─Conv2d: 3-4                       [1, 16, 48, 48]           160
│    │    └─BatchNorm2d: 3-5                  [1, 16, 48, 48]           32
│    │    └─SqueezeExciteModule: 3-6          [1, 16, 1, 1]             544
│    │    └─Conv

In [13]:
# Test MobileNetV3-Small-1.0-160
summary(MobileNetV3(size='small', width_mult=1.0, classifier=False), input_size=(1, 3, 160, 160))

Layer (type:depth-idx)                        Output Shape              Param #
MobileNetV3                                   --                        --
├─Sequential: 1-1                             [1, 576, 5, 5]            --
│    └─Conv2d: 2-1                            [1, 16, 80, 80]           448
│    └─BatchNorm2d: 2-2                       [1, 16, 80, 80]           32
│    └─Hardswish: 2-3                         [1, 16, 80, 80]           --
│    └─Bottleneck: 2-4                        [1, 16, 40, 40]           --
│    │    └─Conv2d: 3-1                       [1, 16, 80, 80]           272
│    │    └─BatchNorm2d: 3-2                  [1, 16, 80, 80]           32
│    │    └─ReLU: 3-3                         [1, 16, 80, 80]           --
│    │    └─Conv2d: 3-4                       [1, 16, 40, 40]           160
│    │    └─BatchNorm2d: 3-5                  [1, 16, 40, 40]           32
│    │    └─SqueezeExciteModule: 3-6          [1, 16, 1, 1]             544
│    │    └─Conv

In [14]:
# Test MobileNetV3-Small-1.0-128
summary(MobileNetV3(size='small', width_mult=1.0, classifier=False), input_size=(1, 3, 128, 128))

Layer (type:depth-idx)                        Output Shape              Param #
MobileNetV3                                   --                        --
├─Sequential: 1-1                             [1, 576, 4, 4]            --
│    └─Conv2d: 2-1                            [1, 16, 64, 64]           448
│    └─BatchNorm2d: 2-2                       [1, 16, 64, 64]           32
│    └─Hardswish: 2-3                         [1, 16, 64, 64]           --
│    └─Bottleneck: 2-4                        [1, 16, 32, 32]           --
│    │    └─Conv2d: 3-1                       [1, 16, 64, 64]           272
│    │    └─BatchNorm2d: 3-2                  [1, 16, 64, 64]           32
│    │    └─ReLU: 3-3                         [1, 16, 64, 64]           --
│    │    └─Conv2d: 3-4                       [1, 16, 32, 32]           160
│    │    └─BatchNorm2d: 3-5                  [1, 16, 32, 32]           32
│    │    └─SqueezeExciteModule: 3-6          [1, 16, 1, 1]             544
│    │    └─Conv

In [15]:
# Test MobileNetV3-Small-1.0-96
summary(MobileNetV3(size='small', width_mult=1.0, classifier=False), input_size=(1, 3, 96, 96))

Layer (type:depth-idx)                        Output Shape              Param #
MobileNetV3                                   --                        --
├─Sequential: 1-1                             [1, 576, 3, 3]            --
│    └─Conv2d: 2-1                            [1, 16, 48, 48]           448
│    └─BatchNorm2d: 2-2                       [1, 16, 48, 48]           32
│    └─Hardswish: 2-3                         [1, 16, 48, 48]           --
│    └─Bottleneck: 2-4                        [1, 16, 24, 24]           --
│    │    └─Conv2d: 3-1                       [1, 16, 48, 48]           272
│    │    └─BatchNorm2d: 3-2                  [1, 16, 48, 48]           32
│    │    └─ReLU: 3-3                         [1, 16, 48, 48]           --
│    │    └─Conv2d: 3-4                       [1, 16, 24, 24]           160
│    │    └─BatchNorm2d: 3-5                  [1, 16, 24, 24]           32
│    │    └─SqueezeExciteModule: 3-6          [1, 16, 1, 1]             544
│    │    └─Conv

## MobileNetV3-Large
- Backbone output (for each 1.0, 0.75, 0.5, 0.35): [960, 7, 7], [960, 6, 6], [960, 5, 5], [960, 4, 4], [960, 3, 3]

## MobileNetV3-Small
- Backbone output (for each 1.0, 0.75, 0.5, 0.35): [576, 7, 7], [576, 6, 6], [576, 5, 5], [576, 4, 4], [576, 3, 3]